#bert 3 

In [ ]:

!pip install transformers==3.0.2
import tensorflow as tf
import pickle
import torch.nn as nn
import torch
from torch.nn.utils.rnn import pack_padded_sequence
from nltk.translate.bleu_score import corpus_bleu
#from processData import Vocabulary
from tqdm import tqdm_notebook as tqdm
import torchvision.models as models
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from PIL import Image
import matplotlib.image as mpimg
from IPython import display
from torchtext.vocab import Vectors, GloVe
import torch
import transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")


In [ ]:


from pickle import dump
from pickle import load
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from keras.preprocessing.text import Tokenizer
from collections import Counter
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from tensorflow.keras.utils import plot_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model


In [ ]:

import pickle 
with open("/content/drive/MyDrive/IMAGE2TWEET/train_img_vgg19.pkl","rb") as f:
  i_e=pickle.load(f)

In [ ]:
import pandas as pd
df1=pd.read_csv("/content/drive/MyDrive/IMAGE2TWEET/english_train_data.csv")


In [ ]:
df1

,Tweet ID,Image URL,Text,Cluster ID
0,1263440694305124352,http://pbs.twimg.com/media/EYikiz5UYAAD6Mu.jpg,#CycloneAmphan: 72 people killed in West Benga...,1
1,1237032071123496962,http://pbs.twimg.com/media/ESrSBkcUwAAnfZ-.jpg,Congress party postpones their 'Gandhi Sandesh...,2
2,1237025347306225670,http://pbs.twimg.com/media/ESqhtA_U8AEwrMQ.jpg,Congress general secretary Mukul Wasnik marrie...,3
3,1237025194767601665,http://pbs.twimg.com/media/ESrL2MYUcAA2eNA.jpg,Techie tested positive for coronavirus in Beng...,4
4,1237024936415211520,http://pbs.twimg.com/media/ESrLnrUUMAITOV6.jpg,Man who returned from Italy confirmed positive...,5
...,...,...,...,...
48787,1148971878783963137,http://pbs.twimg.com/media/D_H307DU0AUts5P.jpg,#WC2019WithTimes #INDvsNZ #INDvNZ #CWC19 #INDv...,28469
48788,1148904697853464577,http://pbs.twimg.com/media/D_GowoEUYAAtVXa.jpg,Deposits in Jan Dhan accounts cross Rs 1 lakh ...,28470
48789,1148889598396944385,http://pbs.twimg.com/media/D_GciluUwAU7XmU.png,.@RahulGandhi's follower count on Twitter cros...,28471
48790,1148788937110016001,http://pbs.twimg.com/media/D_FMNhgU0AAIt-1.jpg,Concerns over government's facial recognition ...,28472


In [ ]:
a=list(i_e.keys())
x=df1.columns
q=list(df1['Text'])
caps={}
for i in a:
  j=int(i.split("_")[1])
  caps[i]=['startseq ' + q[j]+ ' endseq']
      

In [ ]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                se= [word for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    encoded = tokenizer.batch_encode_plus(
                                                    [" ".join(se[:i])],
                                                add_special_tokens=True,
                                                  max_length=61,
                                                return_attention_mask=True,
                                                return_token_type_ids=True,
                                                 pad_to_max_length=True,
                                                   return_tensors="tf",
                                                     truncation=True
                                                )
                    input_ids = np.array(encoded["input_ids"], dtype="int32")
                    attention_masks = np.array(encoded["attention_mask"], dtype="int32")
                    token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
                    sequence_output, pooled_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
                    b1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(sequence_output)
                    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(b1)
                    max_pool = tf.keras.layers.GlobalMaxPooling1D()(b1)
                    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
                    dropout = tf.keras.layers.Dropout(0.3)(concat)

                    # split into input and output pair
                    out_seq = seq[i]
                    # pad input sequence
                    #in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(dropout)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                yield ([np.array(X1), np.array(X2)], np.array(y))
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in list(caps.values()):
    nsents += 1
    for w in sent[0].split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('Vocabulary = %d' % (len(vocab)))


Vocabulary = 9587


In [ ]:
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1

In [ ]:
caption_max_length = 33
batch_size = 1
max_length = 33


In [ ]:
generator = data_generator(caps, i_e, wordtoix, max_length, batch_size)


In [ ]:
epochs = 3
batch_size = 3
steps = len(caps)//batch_size


In [ ]:

from keras.layers.merge import concatenate
from keras.models import Model
import keras.backend as K




inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)


inputs2 = Input(shape=(256,))

decoder1 = add([fe2,inputs2])#add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()


model.compile(loss='categorical_crossentropy', optimizer='adam')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 dropout_124 (Dropout)          (None, 4096)         0           ['input_3[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 256)          1048832     ['dropout_124[0][0]']            
                                                                                                  
 input_4 (InputLayer)           [(None, 256)]        0           []                               
                                                                                            

In [ ]:
model.fit_generator(generator, epochs=20, steps_per_epoch=steps, verbose=1)


In [ ]:
%cd /content/drive/MyDrive/IMAGE2TWEET/

/content/drive/.shortcut-targets-by-id/11Sw-O0lrFYAF-DUvpGN2x_IdU7YnPfZG/IMAGE2TWEET


In [ ]:
from tensorflow.keras.models import save_model

save_model(
    model,
    'models/bert_vgg19_IM2TW.hdf5',
    overwrite=True,
    include_optimizer=True
)



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        #sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        #sequence = pad_sequences([sequence], maxlen=max_length)
        #add bert here
        encoded = tokenizer.batch_encode_plus(
                                                    [in_text],
                                                add_special_tokens=True,
                                                  max_length=61,
                                                return_attention_mask=True,
                                                return_token_type_ids=True,
                                                 pad_to_max_length=True,
                                                   return_tensors="tf",
                                                     truncation=True
                                                )
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
        sequence_output, pooled_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
        b1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(sequence_output)
        avg_pool = tf.keras.layers.GlobalAveragePooling1D()(b1)
        max_pool = tf.keras.layers.GlobalMaxPooling1D()(b1)
        concat = tf.keras.layers.concatenate([avg_pool, max_pool])
        dropout = tf.keras.layers.Dropout(0.3)(concat)



        yhat = model.predict([photo,dropout], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break

    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final


In [ ]:
with open("/content/drive/MyDrive/IMAGE2TWEET/val_img_vgg19.pkl","rb") as f:
    i_e=pickle.load(f)

In [ ]:

ret=[]
z=list(i_e.values())
for i in range(len(z)):
  print(i)
  image=z[i]
  ret.append(greedySearch(image))
#ret


In [ ]:
df=pd.DataFrame(ret,columns=['caps'])
df.to_excel("/content/drive/MyDrive/IMAGE2TWEET/exp_bert_caps.xlsx")

In [ ]:
!pip install pycocoevalcap
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge

!pip install nltk==3.5
import nltk
nltk.download('wordnet')


In [ ]:
class I2TTokeninser:
  def tokenise(self,captions):
      id=[k for k,v in captions.items()]
      PUNCTUATIONS = ["''", "'", "``", "`", "-LRB-", "-RRB-", "-LCB-", "-RCB-",".", "?", "!", ",", ":", "-", "--", "...", ";"]
      caps=[v for k,v in captions.items()] ### [ [caption 1_1,caption 1_2 ,] ,[caption 2_1,caption 2_2,..],..]
      tok_cap=[]
      for i in range(len(caps)):
        temp=[]
        for j in range(len(caps[i])):
          tokenised_caps=' '.join([w for w in caps[i][j].rstrip().split(' ') if w not in PUNCTUATIONS])
          temp.append(tokenised_caps)
        tok_cap.append(temp)
      ret_caps={}
      for i in range(len(id)):
        ret_caps[id[i]]=tok_cap[i]
      return ret_caps
 

    



In [ ]:


def scorer(orig,ret):
  tok=I2TTokeninser()
  ori=tok.tokenise(orig)
  re=tok.tokenise(ret)


  Avg_cider=0.0
  Avg_bleu=0
  Avg_meteor=0
  Avg_rouge=0
  
  for i in range(len(re.keys())):
    temp_re={}
    temp_re[list(re.keys())[i]]=list(re.values())[i]#[list(re.values())[i]]
    caps=ori[list(re.keys())[i]]

    for j in range(len(caps)):
      temp={}
      temp[list(re.keys())[i]]=[caps[j]]

      score,_=Cider().compute_score(temp, temp_re)
      Avg_cider=Avg_cider + score 
    
    Avg_cider= Avg_cider +  Avg_cider/len(caps)   ## getting average score of every ret caption adn referance caption(captions in cluster) and adding to total score(for all retrieved captions)
  Avg_cider=Avg_cider / len(re.keys())
  score, scores = Cider().compute_score(ori, re)
  print("cider ",score)
  Mscore=0
  for i in range(len(re.keys())):
    name=list(re.keys())[i]
    Mscore =Mscore +nltk.translate.meteor_score.meteor_score(ori[name], re[name][0])
  Mscore =Mscore /len(re.keys())
  
  Avg_meteor=Mscore  


  score,_=Rouge().compute_score(ori, re)
  Avg_rouge=score

  BLEUscore = 0
  for i in range(len(re.keys())):
    name=list(re.keys())[i]
    #x=[]
    #for j in range(len(ori[name])):
    #  x.append(ori[name][j].split())
    #BLEUscore =BLEUscore +nltk.translate.bleu_score.sentence_bleu(x, re[name][0].split())
    BLEUscore =BLEUscore +nltk.translate.bleu_score.sentence_bleu(ori[name], re[name][0])
  BLEUscore =BLEUscore /len(re.keys())
  Avg_bleu=BLEUscore
  #print("CIDER: ","{0:.4f}".format(Avg_cider))
  print("Meteor : ",Avg_rouge,"ROUGE : " ,Avg_rouge,"BLEU 4 :  " ,Avg_bleu) # return score 
import pandas as pd

def evaluator(ret):
  #ret === > {image_id : [caption], } example : ret={1300483489071063042:['black animal'],1300465872838950912:['dinner at a restraunt']}

  
  
  df1=pd.read_csv("/content/drive/MyDrive/IMAGE2TWEET/english_test_data (1).csv")







  id=[k for k,v in ret.items()]
  #caps=[v for k,v in ret.items()] 
  orig={}
  for i in range(len(id)):
    cluster_id=list(df1[df1['Tweet ID']==id[i]]['Cluster ID'])[0]
    c=df1[df1['Cluster ID']==cluster_id]['Text']
    temp=[]
    for j in c:
      temp.append(j)
    orig[id[i]]=temp
  #print(orig)
  #print(ret)
  scorer(orig,ret)


In [ ]:
y=ret
ret={}
df2=pd.read_csv("/content/drive/MyDrive/IMAGE2TWEET/english_test_data (1).csv")

x=df2['id']

for i in range(0,len(x)):
  ret[x[i]]=[y[i]]

In [ ]:
evaluator(ret)

cider  3.756380705535758e-04
Meteor :  0.00013100512749380385 ROUGE :  0.00013100512749380385 BLEU 4 :   0.023742007300135464


#bert 3 Hindi

In [ ]:

!pip install transformers==3.0.2
import tensorflow as tf
import pickle
import torch.nn as nn
import torch
from torch.nn.utils.rnn import pack_padded_sequence
from nltk.translate.bleu_score import corpus_bleu
#from processData import Vocabulary
from tqdm import tqdm_notebook as tqdm
import torchvision.models as models
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from PIL import Image
import matplotlib.image as mpimg
from IPython import display
from torchtext.vocab import Vectors, GloVe
import torch
import transformers
from transformers import BertTokenizer, BertModel, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")


In [ ]:


from pickle import dump
from pickle import load
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from keras.preprocessing.text import Tokenizer
from collections import Counter
from tensorflow.keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from tensorflow.keras.utils import plot_model
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model


In [ ]:

import pickle 
with open("/content/drive/MyDrive/IMAGE2TWEET/hindi_train_img_vgg19.pkl","rb") as f:
  i_e=pickle.load(f)

In [ ]:
import pandas as pd
df1=pd.read_csv("/content/drive/MyDrive/IMAGE2TWEET/hindi_train_data.csv")


In [ ]:
df1

,Tweet ID,Image URL,Text,Cluster ID
0,765816844875800576,http://pbs.twimg.com/media/CqC6IKaUIAEk81F.jpg,विदेश नीति में बड़ा बदलाव? #US को खुश करने के ...,1
1,765801793355911168,http://pbs.twimg.com/media/CqCsiARXgAAUFu-.jpg,#Ladakh में 14000 फीट ऊपर मिली 10000 साल पुरान...,2
2,765801486043508736,http://pbs.twimg.com/media/CqCr2m-WgAQQVvz.jpg,"#Bihar में जहरीली शराब पीने से 13 की मौत, 4 मह...",3
3,765787288068616192,http://pbs.twimg.com/media/CqCfPl0XEAEcr8l.jpg,#maharashtra के दबंग MLA ने कलेक्टर को मारे थप...,4
4,765779586496000000,http://pbs.twimg.com/media/CqCYHz7WEAARY8v.jpg,#China के खिलाफ #भारत ने बढ़ाई ताकत: लद्दाख मे...,5
...,...,...,...,...
35696,607047306739777536,http://pbs.twimg.com/media/CGyqTrDUYAM7TkD.jpg,"रोचक है टेलीविजन का इतिहास, इन बातों को शायद आ...",32865
35697,607047239203094528,http://pbs.twimg.com/media/CGyqPxNUAAAc2ed.jpg,"रोचक है टेलीविजन का इतिहास, इन बातों को शायद आ...",32866
35698,607047059103825920,http://pbs.twimg.com/media/CGyqFSZVIAAE5wF.jpg,"रोचक है टेलीविजन का इतिहास, इन बातों को शायद आ...",32867
35699,606724304596631552,http://pbs.twimg.com/media/CGuEcoTUQAA-KuF.jpg,"MONSOON: मानसून ने केरल में दी दस्तक, अगले दो ...",32868


In [ ]:
a=list(i_e.keys())
x=df1.columns
q=list(df1['Text'])
caps={}
for i in a:
  j=int(i.split("_")[1])
  caps[i]=['startseq ' + q[j]+ ' endseq']
      

In [ ]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n+=1
            # retrieve the photo feature
            photo = photos[key]
            for desc in desc_list:
                # encode the sequence
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                se= [word for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    encoded = tokenizer.batch_encode_plus(
                                                    [" ".join(se[:i])],
                                                add_special_tokens=True,
                                                  max_length=61,
                                                return_attention_mask=True,
                                                return_token_type_ids=True,
                                                 pad_to_max_length=True,
                                                   return_tensors="tf",
                                                     truncation=True
                                                )
                    input_ids = np.array(encoded["input_ids"], dtype="int32")
                    attention_masks = np.array(encoded["attention_mask"], dtype="int32")
                    token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
                    sequence_output, pooled_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
                    b1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(sequence_output)
                    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(b1)
                    max_pool = tf.keras.layers.GlobalMaxPooling1D()(b1)
                    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
                    dropout = tf.keras.layers.Dropout(0.3)(concat)

                    # split into input and output pair
                    out_seq = seq[i]
                    # pad input sequence
                    #in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(dropout)
                    y.append(out_seq)

            if n==num_photos_per_batch:
                #yield ([np.array(X1), np.array(X2)], np.array(y))
                yield ([np.array(X1).reshape(-1,4096), np.array(X2).reshape(-1,256)], np.array(y).reshape(-1,7334))
                X1, X2, y = list(), list(), list()
                n=0

In [ ]:
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in list(caps.values()):
    nsents += 1
    for w in sent[0].split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1
vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

print('Vocabulary = %d' % (len(vocab)))


Vocabulary = 7333


In [ ]:
ixtoword = {}
wordtoix = {}
ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

vocab_size = len(ixtoword) + 1

In [ ]:
caption_max_length = 33
batch_size = 1
max_length = 33


In [ ]:
generator = data_generator(caps, i_e, wordtoix, max_length, batch_size)


In [ ]:
epochs = 3
batch_size = 3
steps = len(caps)//batch_size


In [ ]:

from keras.layers.merge import concatenate
from keras.models import Model
import keras.backend as K





inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)


inputs2 = Input(shape=(256,))


decoder1 = add([fe2,inputs2])#add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()


model.compile(loss='categorical_crossentropy', optimizer='adam')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 dropout_37 (Dropout)           (None, 4096)         0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 256)          1048832     ['dropout_37[0][0]']             
                                                                                                  
 input_2 (InputLayer)           [(None, 256)]        0           []                               
                                                                                              

In [ ]:
model.fit_generator(generator, epochs=20, steps_per_epoch=steps, verbose=1)


In [ ]:
%cd /content/drive/MyDrive/IMAGE2TWEET/

/content/drive/.shortcut-targets-by-id/11Sw-O0lrFYAF-DUvpGN2x_IdU7YnPfZG/IMAGE2TWEET


In [ ]:
from tensorflow.keras.models import save_model

save_model(
    model,
    'models/HINDI_bert_vgg19_IM2TW.hdf5',
    overwrite=True,
    include_optimizer=True
)



/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        #sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        #sequence = pad_sequences([sequence], maxlen=max_length)
        #add bert here
        encoded = tokenizer.batch_encode_plus(
                                                    [in_text],
                                                add_special_tokens=True,
                                                  max_length=61,
                                                return_attention_mask=True,
                                                return_token_type_ids=True,
                                                 pad_to_max_length=True,
                                                   return_tensors="tf",
                                                     truncation=True
                                                )
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
        sequence_output, pooled_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
        b1=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(sequence_output)
        avg_pool = tf.keras.layers.GlobalAveragePooling1D()(b1)
        max_pool = tf.keras.layers.GlobalMaxPooling1D()(b1)
        concat = tf.keras.layers.concatenate([avg_pool, max_pool])
        dropout = tf.keras.layers.Dropout(0.3)(concat)



        yhat = model.predict([photo,dropout], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break

    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final


In [ ]:
with open("/content/drive/MyDrive/IMAGE2TWEET/hindi_test_img_vgg19.pkl","rb") as f:
    i_e=pickle.load(f)

In [ ]:
ret=[]
z=list(i_e.values())
for i in range(len(z)):
  print(i)
  image=z[i]
  ret.append(greedySearch(image))


In [ ]:
!pip install pycocoevalcap
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge

!pip install nltk==3.5
import nltk
nltk.download('wordnet')


In [ ]:
class I2TTokeninser:
  def tokenise(self,captions):
      id=[k for k,v in captions.items()]
      PUNCTUATIONS = ["''", "'", "``", "`", "-LRB-", "-RRB-", "-LCB-", "-RCB-",".", "?", "!", ",", ":", "-", "--", "...", ";"]
      caps=[v for k,v in captions.items()] ### [ [caption 1_1,caption 1_2 ,] ,[caption 2_1,caption 2_2,..],..]
      tok_cap=[]
      for i in range(len(caps)):
        temp=[]
        for j in range(len(caps[i])):
          tokenised_caps=' '.join([w for w in caps[i][j].rstrip().split(' ') if w not in PUNCTUATIONS])
          temp.append(tokenised_caps)
        tok_cap.append(temp)
      ret_caps={}
      for i in range(len(id)):
        ret_caps[id[i]]=tok_cap[i]
      return ret_caps
 

    



In [ ]:


def scorer(orig,ret):
  tok=I2TTokeninser()
  ori=tok.tokenise(orig)
  re=tok.tokenise(ret)


  Avg_cider=0.0
  Avg_bleu=0
  Avg_meteor=0
  Avg_rouge=0
  
  for i in range(len(re.keys())):
    temp_re={}
    temp_re[list(re.keys())[i]]=list(re.values())[i]#[list(re.values())[i]]
    caps=ori[list(re.keys())[i]]

    for j in range(len(caps)):
      temp={}
      temp[list(re.keys())[i]]=[caps[j]]

      score,_=Cider().compute_score(temp, temp_re)
      Avg_cider=Avg_cider + score 
    
    Avg_cider= Avg_cider +  Avg_cider/len(caps)   ## getting average score of every ret caption adn referance caption(captions in cluster) and adding to total score(for all retrieved captions)
  Avg_cider=Avg_cider / len(re.keys())
  score, scores = Cider().compute_score(ori, re)
  print("cider ",score)
  Mscore=0
  for i in range(len(re.keys())):
    name=list(re.keys())[i]
    Mscore =Mscore +nltk.translate.meteor_score.meteor_score(ori[name], re[name][0])
  Mscore =Mscore /len(re.keys())
  
  Avg_meteor=Mscore  


  score,_=Rouge().compute_score(ori, re)
  Avg_rouge=score

  BLEUscore = 0
  for i in range(len(re.keys())):
    name=list(re.keys())[i]
    #x=[]
    #for j in range(len(ori[name])):
    #  x.append(ori[name][j].split())
    #BLEUscore =BLEUscore +nltk.translate.bleu_score.sentence_bleu(x, re[name][0].split())
    BLEUscore =BLEUscore +nltk.translate.bleu_score.sentence_bleu(ori[name], re[name][0])
  BLEUscore =BLEUscore /len(re.keys())
  Avg_bleu=BLEUscore
  #print("CIDER: ","{0:.4f}".format(Avg_cider))
  print("Meteor : ",Avg_rouge,"ROUGE : " ,Avg_rouge,"BLEU 4 :  " ,Avg_bleu) # return score 
import pandas as pd

def evaluator(ret):
  #ret === > {image_id : [caption], } example : ret={1300483489071063042:['black animal'],1300465872838950912:['dinner at a restraunt']}

  df1=pd.read_csv("/content/drive/MyDrive/IMAGE2TWEET/hindi_test_data (1).csv")







  id=[k for k,v in ret.items()]
  #caps=[v for k,v in ret.items()] 
  orig={}
  for i in range(len(id)):
    cluster_id=list(df1[df1['Tweet ID']==id[i]]['Cluster ID'])[0]
    c=df1[df1['Cluster ID']==cluster_id]['Text']
    temp=[]
    for j in c:
      temp.append(j)
    orig[id[i]]=temp
  #print(orig)
  #print(ret)
  scorer(orig,ret)


In [ ]:
y=ret
r={}


In [ ]:
df2=pd.read_csv("/content/drive/MyDrive/IMAGE2TWEET/hindi_test_data (1).csv")
e=df2.columns

x=df2[e[0]]


In [ ]:
df2

,Tweet ID,Image URL,Text,Cluster ID
0,672060201063714816,http://pbs.twimg.com/media/CVOjMMFUsAAzuu1.jpg,Today's Headlines@ 8 PM\r\nरिपोर्ट में दावा: I...,1
1,672035790541598720,http://pbs.twimg.com/media/CVOM_VQUwAABF3k.jpg,‘बाजीराव-मस्तानी’ के नए पोस्टर में दिखा दीपिका...,2
2,672029674831110144,http://pbs.twimg.com/media/CVOHbWRVEAEpvKv.jpg,क्रिमिनल्स के छक्के छुड़ाने वाले इस CSP ने रिट...,3
3,672017633508638721,http://pbs.twimg.com/media/CVN8eaYVEAAH9_d.jpg,दुनिया के सबसे यंगेस्ट खरबपति की जिंदगी में बे...,4
4,672009054810562560,http://pbs.twimg.com/media/CVN0q_GUsAATffq.jpg,एक विस्फोट...और देखते ही देखते हवा में उड़े ची...,5
...,...,...,...,...
7647,877044934351568896,http://pbs.twimg.com/media/DCvjhwMUQAA4nK_.jpg,1993 बॉम्बे ब्लास्ट: CBI ने सलेम को छोड़कर 5 द...,7043
7648,914537416190578688,http://pbs.twimg.com/media/DLEWycXUQAAWJTT.jpg,"'गोरखपुर को जल्द मिलेगा इंटरनेशनल स्टेडियम, 5 ...",7043
7649,1098199948221243393,http://pbs.twimg.com/media/Dz2XBCwUYAUal4f.jpg,कंफर्म / क्वालकॉम ने बनाई दूसरी पीढ़ी की 5 जी ...,7043
7650,1122845186210459648,http://pbs.twimg.com/media/D5UlsapU0AEFIAp.jpg,लोकसभा चुनाव / 5 बजे तक 54% वोटिंग: अफसर को धम...,7043


In [ ]:

for i in range(0,len(y)):
  r[x[i]]=[y[i]]  

In [ ]:
import nltk

evaluator(r)


cider  4.958281440697747e-04 
 Meteor :  0.0023126010193659234 ROUGE :  0.00023126010193659234 BLEU 4 :   0.033913476601767118
